<a href="https://colab.research.google.com/github/rvinacio/establishment-classifier-gemini/blob/main/categorizador_inteligente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [36]:
# Importar as bibliotecas do google

import google.generativeai as genai
from google.colab import userdata
from google.api_core import retry

GOOGLE_API_KEY=userdata.get('key_standard')
genai.configure(api_key=GOOGLE_API_KEY)

In [37]:
# Listar modelos disponíveis

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [38]:
# Configurar níveis de "criatividade" do modelo
generation_config = {
  "temperature": 0.55,
  "top_p": 1,
  "top_k": 0,
}

In [39]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_ONLY_HIGH"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_ONLY_HIGH"
  },
]

In [40]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


In [76]:
prompt_parts = [
  "Você é um analista de dados especializado em categorizar estabelecimentos comerciais. Sua tarefa é analisar o nome do estabelecimento e classificá-lo na categoria mais adequada, fornecendo uma breve justificativa baseada em padrões e palavras-chave relevantes. Se a categoria for ambígua, utilize \"SEM CATEGORIA\".",
  "input: Null",
  "output: Sem Categoria",
  "output_justificativa Não foi possível categorizar",
  "input: Julio Cesar Soares Da",
  "output: Sem Categoria",
  "output_justificativa Não foi possível categorizar",
  "input: Andre Conrado Fer",
  "output: Sem Categoria",
  "output_justificativa Não foi possível categorizar",
  "input: Dm*Udemy",
  "output: Educação",
  "output_justificativa Trata-se de um curso",
  "input: Iso Radiologia Di",
  "output: Saúde",
  "output_justificativa Clínica de exames odontológicos",
  "input: Clinica*Chammas Odont",
  "output: Saúde",
  "output_justificativa Clínica odontológica",
  "input: Mp*Taxi",
  "output: Transporte",
  "output_justificativa Corrida de taxi",
  "input: Metro Rj",
  "output: Transporte",
  "output_justificativa Metro",
  "input: Uber",
  "output: Transporte",
  "output_justificativa Uber",
  "input: Beleza Na Web",
  "output: DermoCosmético",
  "output_justificativa Loja de dermocosméticos",
  "input: Sephora Dotcom",
  "output: DermoCosmético",
  "output_justificativa Perfumaria",
  "input: Posto 011 - Vila Maria",
  "output: Atividade Física",
  "output_justificativa Esportes de areia",
  "input: Bayard Esportes",
  "output: Atividade Física",
  "output_justificativa Loja de produtos de esporte",
  "input: Totalpass",
  "output: Atividade Física",
  "output_justificativa Academia/Musculação",
  "input: Daki",
  "output: Mercado",
  "output_justificativa Mercado",
  "input: Master Supermercados",
  "output: Mercado",
  "output_justificativa Supermercado",
  "input: Carrefour Epl",
  "output: Mercado",
  "output_justificativa Supermercado",
  "input: Apple.Com/Bill",
  "output: Serviços",
  "output_justificativa Pagamento do icloud",
  "input: Mp*Manicure10",
  "output: Serviços",
  "output_justificativa Serviço de manicure",
  "input: Vitor Pimenta Matos 499",
  "output: Serviços",
  "output_justificativa Serviço de barbearia",
  "input: John John",
  "output: Vestuário",
  "output_justificativa Loja de roupas",
  "input: Mash Online",
  "output: Vestuário",
  "output_justificativa Lojas de cuecas",
  "input: Amazon Br",
  "output: Compras",
  "output_justificativa Lojas de produtos eletrônicos e diversos",
  "input: Casasbahia",
  "output: Compras",
  "output_justificativa Lojas de produtos eletrônicos e móveis",
  "input: Koni Brasilia Sho",
  "output: Viagem",
  "output_justificativa Lanchonete em Brasília",
  "input: Braseiro Da Gavea",
  "output: Viagem",
  "output_justificativa Restaurante no Rio de Janeiro",
  "input: Viacao Aguia Branca S.A.",
  "output: Viagem",
  "output_justificativa Passagem rodoviária",
  "input: Booking.Com Brasil Servic",
  "output: Viagem",
  "output_justificativa Hospedagem",
  "input: Ms Live Eventos E Consu",
  "output: Entretenimento",
  "output_justificativa Evento",
  "input: Jerome Bar Ltda",
  "output: Entretenimento",
  "output_justificativa Balada",
  "input: Zp     *Sympla 5mpa0q",
  "output: Entretenimento",
  "output_justificativa Ingresso",
  "input: Aue Bar E Restaurante",
  "output: Entretenimento",
  "output_justificativa Bar",
  "input: Funilaria Bixiga",
  "output: Entretenimento",
  "output_justificativa Samba",
  "input: Sesc Pompeia",
  "output: Entretenimento",
  "output_justificativa Show",
  "input: Margherita Pizzeria",
  "output: Alimentação",
  "output_justificativa Pizzaria",
  "input: Macaxeira",
  "output: Alimentação",
  "output_justificativa Restaurante",
  "input: Cafe E Lanchonete Miyas",
  "output: Alimentação",
  "output_justificativa Cafeteria",
  "input: Perola Da Vila Mariana",
  "output: Alimentação",
  "output_justificativa Padaria",
  "input: Brigaderia Da Vovo",
  "output: Alimentação",
  "output_justificativa Doceria",
  "input: Ifood",
  "output: Alimentação",
  "output_justificativa Compra app ifood",
]

In [72]:
# Função para formatar o prompt a partir de prompt_parts e novos inputs (inalterada)
def format_prompt_with_new_inputs(prompt_parts, new_inputs):
    prompt = "\n".join(prompt_parts)
    for input_text in new_inputs:
        prompt += f"\ninput: {input_text}"
        # Imprime a entrada do estabelecimento
        print(f"input: {input_text}")
    return prompt

# Recebe os nomes dos estabelecimentos (uma única vez)
new_inputs = input("Digite os nomes dos estabelecimentos (separados por vírgula): ").split(",")

# Formatação do prompt com os novos inputs
prompt = format_prompt_with_new_inputs(prompt_parts, new_inputs)

# Geração de conteúdo (com tratamento de erros)
@retry.Retry()
def generate_with_retry(prompt):
    response = model.generate_content(prompt)
    return response

response = generate_with_retry(prompt)


# Processamento da resposta (formato detalhado com tratamento de erros e formato inconsistente)
if response.candidates:
    for candidate in response.candidates:
        text_parts = candidate.content.parts[0].text.split("\n")
        print(text_parts)  # Imprime o conteúdo de text_parts para análise

        i = 0
        while i < len(text_parts):
            if text_parts[i].startswith("input:"):
                estabelecimento = text_parts[i].split(": ")[1]
                categoria = "Sem Categoria"
                justificativa = "Justificativa não fornecida"

                # Procura pelas linhas "output:" e "output_justificativa:"
                while i + 1 < len(text_parts) and not text_parts[i + 1].startswith("input:"):
                    if text_parts[i + 1].startswith("output:"):
                        categoria = text_parts[i + 1].split(": ")[1]
                    elif text_parts[i + 1].startswith("output_justificativa:"):
                        justificativa = text_parts[i + 1].split(": ")[1]
                    i += 1

                print(f"input: {estabelecimento}")
                print(f"output: {categoria}")
                print(f"output_justificativa: {justificativa}")
                i += 1  # Avança para a próxima linha
            else:
                i += 1
else:
    print("Nenhuma resposta gerada.")



Digite os nomes dos estabelecimentos (separados por vírgula): uber, ifood, hopital santa maria
input: uber
input:  ifood
input:  hopital santa maria
['output: Transporte', 'output_justificativa Corrida de taxi', 'output: Alimentação', 'output_justificativa Compra app ifood', 'output: Saúde', 'output_justificativa Hospital']
